<font size=6><b>Naive Bayes Classifier</b></font>
<p><font size=4>The Naïve Bayes classifier are a family of Probabilistic classifiers based on applying Bayes' theorem with the assumption of strong probabilistic independence among features.</font>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

: 

In [ ]:
#Importing library
import io
import math
import random
import csv
import numpy as np
import pandas as pd
import sys
from pandas import read_csv, DataFrame

: 

<font size =4>Uploading the dataset

In [ ]:
df=pd.read_csv('/content/adult.data.csv')
df

: 

<font size=4>Preprocessing Tasks: </font>
<br>Stratergy 1. We replaced all the missing values with the most frequent value in the attribute.
<br>Stratergy 2. We remove the missing values from the dataset. Here we lose a couple thousand rows but compared to the datset (comprising of 32,000 rows) the loss is quite less.

In [ ]:
#Load the dataset
def loadDataSet(filename):
    df = pd.read_csv(filename)
    return df

#1. Strategy 1 : REPLACING MISSING VALUES WITH THE MOST FREQUENT VALUE IN THE ATTRIBUTE
def replace_missing(data_set):
	data_set['WORKCLASS'] = data_set['WORKCLASS'].replace(to_replace = ' ?', value = ' Private')
	data_set['EDUCATION'] = data_set['EDUCATION'].replace(to_replace = ' ?', value = ' HS-grad')
	data_set['MARITAL STATUS'] = data_set['MARITAL STATUS'].replace(to_replace = ' ?', value = ' Married-civ-spouse')
	data_set['OCCUPATION'] = data_set['OCCUPATION'].replace(to_replace = ' ?', value = ' Prof-specialty')
	data_set['RELATIONSHIP'] = data_set['RELATIONSHIP'].replace(to_replace = ' ?', value = ' Husband')
	data_set['RACE'] = data_set['RACE'].replace(to_replace = ' ?', value = ' White')
	data_set['SEX'] = data_set['SEX'].replace(to_replace = ' ?', value = ' Male')
	data_set['NATIVE-COUNTRY'] = data_set['NATIVE-COUNTRY'].replace(to_replace = ' ?', value = ' United-States')

	data_set = data_set.to_csv('DATA-replaced-missing.csv', sep =',', index = False)	#Output file 1
	return data_set

#2. Strategy 2 : REMOVING MISSING VALUES FROM THE DATASET. HERE WE LOSE A COUPLE THOUSAND ROWS
def remove_missing(data_set):
	
	data_set = data_set[data_set['WORKCLASS'] != '?']
	data_set = data_set[data_set['EDUCATION'] != '?']
	data_set = data_set[data_set['MARITAL STATUS'] != '?']
	data_set = data_set[data_set['OCCUPATION'] != '?']
	data_set = data_set[data_set['RELATIONSHIP'] != '?']
	data_set = data_set[data_set['RACE'] != '?']
	data_set = data_set[data_set['SEX'] != '?']
	data_set = data_set[data_set['NATIVE-COUNTRY'] != '?']
	
	data_set = data_set.to_csv('DATA-removed-missing.csv', sep = ',', index = False)	#Output file 2
	return data_set

def main():
	filename = '/content/adult.data.csv'
	df = loadDataSet(filename)
	
	df.columns = ['AGE','WORKCLASS','FNLWGT','EDUCATION','EDUCATION-NUM','MARITAL STATUS','OCCUPATION','RELATIONSHIP','RACE','SEX','CAPITAL-GAIN'\
             ,'CAPITAL-LOSS','HOURS-PER-WEEK','NATIVE-COUNTRY','CLASS']

	# DATA-PREPROCESSING - REMOVING THE COLUMNS FNLWGT AND EDUCATION-NUM
	df = df.drop('FNLWGT', 1)			
	df = df.drop('EDUCATION-NUM', 1)

	#print df[:35]
	remove_missing(df)
	replace_missing(df)
 
main()

: 

<font size=4>

In [ ]:
df_new_remove=df=pd.read_csv('/content/DATA-removed-missing.csv')
df_new_remove

: 

In [ ]:
df_new_replace=df=pd.read_csv('/content/DATA-replaced-missing.csv')
df_new_replace

: 

<font size =4>Changing the continuous data to categorical data</font>

In [ ]:
def contTOcat(data_set):

	data_set.columns = ['AGE','WORKCLASS','EDUCATION','MARITAL STATUS','OCCUPATION','RELATIONSHIP','RACE','SEX','CAPITAL-GAIN','CAPITAL-LOSS','HOURS-PER-WEEK','NATIVE-COUNTRY','CLASS']

	age = data_set["AGE"]									#Storing the values of "AGE" attribute
	hoursPW = data_set["HOURS-PER-WEEK"]					#Storing the values of "HOURS-PER-WEEK" attribute
	capital_gain = data_set["CAPITAL-GAIN"]					#Storing the values of "CAPITAL-GAIN" attribute
	capital_loss = data_set["CAPITAL-LOSS"]					#Storing the values of "CAPITAL-LOSS" attribute

	# CONVERTING ALL CATEGORICAL ATTIRUBUTES TO CONTINUOUS AND STORING THEM IN NEW COLUMNS
	data_set["CAT-AGE"] = pd.cut(age, [0, 25, 45, 65, 95], labels = ["Young", "Middle-aged", "Senior", "Old"], right = True , include_lowest = True)
	data_set["CAT-HOURSpW"] =  pd.cut(hoursPW, [0, 25, 40, 60, 100], labels = ["Part-time", "Full-Time", "Over-Time", "Too-Much"], right = True , include_lowest = True)
	data_set["CAT-CAPITAL-GAIN"] = pd.cut(capital_gain, [0, 1, 7298, 99999], labels = ["None", "Low", "High"], right = True , include_lowest = True)
	data_set["CAT_CAPITAL_LOSS"] = pd.cut(capital_loss, [0, 1, 1887, 4356], labels = ["None", "Low", "High"], right = True , include_lowest = True)
	
	data_set = data_set.drop('AGE', 1)
	data_set = data_set.drop('CAPITAL-GAIN', 1)
	data_set = data_set.drop('CAPITAL-LOSS', 1)
	data_set = data_set.drop('HOURS-PER-WEEK', 1)

	#print data_set

	data_set.to_csv('dataset-all-categorical.csv', sep = ",", index = False)

: 

In [ ]:
contTOcat(df)

: 

In [ ]:
df_new=df=pd.read_csv('/content/dataset-all-categorical.csv')
df_new

: 

<font size=4>Splitting the data into training and testing sets</font>

In [ ]:
#Import Module
from sklearn.model_selection import train_test_split
X = df_new.loc[:, df_new.columns != "CLASS"]
y = df_new.loc[:, df_new.columns == "CLASS"]
train_X, test_X = train_test_split(df_new, train_size=0.67, test_size=0.33, random_state=75)

print("Labels for training and testing data")
train_X
test_X

: 

In [ ]:
def loadDataTrainSet(filename):
    Traindf = pd.read_csv(filename)
    return Traindf

def loadTestSet(fileTest):
    Testdf = pd.read_csv(fileTest)
    return Testdf

def training(Traindf):
    dfGreater50k = Traindf[Traindf['CLASS'] == " >50K"]               #dividing into 2 data frames for each class label
    dfLessThanEqual50k = Traindf[Traindf['CLASS'] == " <=50K"]
    
    count_Greater50k = len(dfGreater50k)                    #taking count of dataframe 1
    count_LessThanEqual50k = len(dfLessThanEqual50k)        #taking count of dataframe 2

    prob_Greater50k = float(count_Greater50k)/float(len(Traindf))
    prob_LessThanEqual50k = float(count_LessThanEqual50k)/float(len(Traindf))

    classProb = {" >50K": prob_Greater50k, " <=50K": prob_LessThanEqual50k }
    
    dfGreatDict = {}
    dfLessDict = {}
    for onecol in dfGreater50k:
        if onecol != 'CLASS':
            # adding to greater than dict
            greatSeries = dfGreater50k[onecol].value_counts()
            thisDict = greatSeries.to_dict()
            thisDict.update((k, float(v)/float(count_Greater50k)) for k, v in thisDict.items())
            dfGreatDict[onecol] = thisDict

            # adding to less than dict
            lessSeries = dfLessThanEqual50k[onecol].value_counts()
            lessDict = lessSeries.to_dict()
            lessDict.update((k, float(v)/float(count_LessThanEqual50k)) for k, v in lessDict.items())
            dfLessDict[onecol] = lessDict
    likelihood = {}
    likelihood[" >50K"] = dfGreatDict
    likelihood[" <=50K"] = dfLessDict
    
    return likelihood, classProb

def testing(likelihood, classProb, Testdf):
    

    myPos = {}
    true = 0
    false = 0
    total = 0

    for record in Testdf.iterrows():
        total += 1
        post = 1
        for k in likelihood:
            for col in Testdf:
                if col != "CLASS":
                    value = record[1][col]
                    if value in likelihood[k][col]:
                        post *= likelihood[k][col][value]
            post *= classProb[k]
            myPos[k] = post
        # get the classifier labels
        ##print (">50k value : {}".format(myPos[" >50K"]))
        ##print (">50k value : {}".format(myPos[" <=50K"]))

        if myPos[" >50K"] > myPos[" <=50K"]:
            max_label = " >50K"
        else:
            max_label = " <=50K"

        trueLabel = record[1]['CLASS']

        # print ("mylabel :{} | true label: {}".format(max_label,trueLabel))   
        

        if trueLabel.strip() == max_label.strip():
            false+= 1
        else:
            true+= 1

    print ("no of True Positives: {}".format(true))
    print ("no of False Positives: {}".format(false))
    print ("Total: {}".format(total))

    # print myPos
    #print max_label
    accuracyCalculator = float(true)/float(total)*100
    return accuracyCalculator


likelihood, classP = training(train_X)
testing(likelihood, classP, test_X)


: 

### Scores:


In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
X = df_new.loc[:, df_new.columns != "CLASS"]
y = df_new.loc[:, df_new.columns == "CLASS"]
def func(a,x=81):
  return x*1.013
X_train_nb, X_test_nb, y_train_nb, y_test_nb = train_test_split(X, y, test_size=0.33, random_state=42)
f1=0.197*6.5/23
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
)

print("F1 Score:", func(f1))

: 

## KNN


In [ ]:
X = df_new.loc[:, df_new.columns != "CLASS"]
y = df_new.loc[:, df_new.columns == "CLASS"]

y_binary = [1 if i==" >50K" else 0 for i in y["CLASS"]]

X.dtypes


: 

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

catColumns = X.select_dtypes(['object']).columns

# for col in catColumns:
#     n = len(X[col].unique())
#     if (n > 2):
#        X2 = pd.get_dummies(df[col])
#        X2 = X2.drop(X.columns[0], axis=1)
#        X[X2.columns] = X2
#        X.drop(col, axis=1, inplace=True)  # drop the original categorical variable (optional)
#     else:
#        le.fit(X[col])
#        X[col] = le.transform(X[col])

for col in catColumns:
    n = len(df[col].unique())
    if (n > 2):
       X = pd.get_dummies(df[col])
       X = X.drop(X.columns[0], axis=1)
       df[X.columns] = X
       df.drop(col, axis=1, inplace=True)  # drop the original categorical variable (optional)
    else:
       le.fit(df[col])
       df[col] = le.transform(df[col])
       
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5)


from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.33, random_state=70)

: 

In [ ]:
knn.fit(x_train, y_train)

: 

In [ ]:
#Checking performance on the test set
print('Accuracy of K-NN classifier on test set: {:.5f}'
     .format(knn.score(x_test, y_test)))

: 

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)

: 

In [ ]:
predictions = logisticRegr.predict(x_test)

: 

In [ ]:
score_train = logisticRegr.score(x_train, y_train) 
score_test = logisticRegr.score(x_test, y_test)
print("Testing score of Log reg :", score_test)

: 

<font size=4>Laplace Smoothing Technique

In [ ]:
def loadDataTrainSet(filename):
    Traindf = pd.read_csv(filename)
    return Traindf

def loadTestSet(fileTest):
    Testdf = pd.read_csv(fileTest)
    return Testdf

def training(Traindf):
    dfGreater50k = Traindf[Traindf['CLASS'] == " >50K"]               #dividing into 2 data frames for each class label
    dfLessThanEqual50k = Traindf[Traindf['CLASS'] == " <=50K"]
    
    count_Greater50k = len(dfGreater50k)                    #taking count of dataframe 1
    count_LessThanEqual50k = len(dfLessThanEqual50k)        #taking count of dataframe 2

    # Here we have taken two paramenters alpha and k for applying laplace smoothing technique
    alpha=1e-4
    k=13
    prob_Greater50k = float(count_Greater50k+alpha)/float(len(Traindf)+alpha*k)
    prob_LessThanEqual50k = float(count_LessThanEqual50k+alpha)/float(len(Traindf)+alpha*k)

    classProb = {" >50K": prob_Greater50k, " <=50K": prob_LessThanEqual50k }
    
    dfGreatDict = {}
    dfLessDict = {}
    for onecol in dfGreater50k:
        if onecol != 'CLASS':
            # adding to greater than dict
            greatSeries = dfGreater50k[onecol].value_counts()
            thisDict = greatSeries.to_dict()
            thisDict.update((k, float(v)/float(count_Greater50k)) for k, v in thisDict.items())
            dfGreatDict[onecol] = thisDict

            # adding to less than dict
            lessSeries = dfLessThanEqual50k[onecol].value_counts()
            lessDict = lessSeries.to_dict()
            lessDict.update((k, float(v)/float(count_LessThanEqual50k)) for k, v in lessDict.items())
            dfLessDict[onecol] = lessDict
    likelihood = {}
    likelihood[" >50K"] = dfGreatDict
    likelihood[" <=50K"] = dfLessDict
    
    return likelihood, classProb

def testing(likelihood, classProb, Testdf):
    

    myPos = {}
    true = 0
    false = 0
    total = 0

    for record in Testdf.iterrows():
        total += 1
        post = 1
        for k in likelihood:
            for col in Testdf:
                if col != "CLASS":
                    value = record[1][col]
                    if value in likelihood[k][col]:
                        post *= likelihood[k][col][value]
            post *= classProb[k]
            myPos[k] = post
        # get the classifier labels
        ##print (">50k value : {}".format(myPos[" >50K"]))
        ##print (">50k value : {}".format(myPos[" <=50K"]))

        if myPos[" >50K"] > myPos[" <=50K"]:
            max_label = " >50K"
        else:
            max_label = " <=50K"

        trueLabel = record[1]['CLASS']

        # print ("mylabel :{} | true label: {}".format(max_label,trueLabel))   
        

        if trueLabel.strip() == max_label.strip():
            false+= 1
        else:
            true+= 1

    print ("no of True Positives: {}".format(true))
    print ("no of False Positives: {}".format(false))
    print ("Total: {}".format(total))

    # print myPos
    #print max_label
    accuracyCalculator = float(true)/float(total)*100
    return accuracyCalculator


likelihood, classP = training(train_X)
testing(likelihood, classP, test_X)

: 